## 1. Configuración de Spark y Carga de Datos

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

# Crear sesión de Spark con configuración optimizada
spark = SparkSession.builder \
    .appName("Olist Exploratory Analysis") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

print("✓ Sesión de Spark creada")
print(f"Versión de Spark: {spark.version}")

✓ Sesión de Spark creada
Versión de Spark: 3.5.0


In [2]:
# Cargar el dataset unificado
df = spark.read.parquet("Data/olist_unified_dataset.parquet")

# Cachear para análisis más rápido
df.cache()

print("=" * 70)
print("DATASET CARGADO")
print("=" * 70)
print(f"Registros: {df.count():,}")
print(f"Columnas: {len(df.columns)}")
print(f"Particiones: {df.rdd.getNumPartitions()}")

DATASET CARGADO
Registros: 112,650
Columnas: 52
Particiones: 6


In [3]:
# Ver esquema del dataset
print("Esquema del dataset:")
df.printSchema()

Esquema del dataset:
root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp_ntz (nullable = true)
 |-- order_approved_at: timestamp_ntz (nullable = true)
 |-- order_delivered_carrier_date: timestamp_ntz (nullable = true)
 |-- order_delivered_customer_date: timestamp_ntz (nullable = true)
 |-- order_estimated_delivery_date: timestamp_ntz (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: long (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- order_item_id: double (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: string (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_nam

In [4]:
# Primeras filas
print("Primeras 10 filas:")
df.show(10, truncate=True)

Primeras 10 filas:
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+--------------------+------------------------+--------------------+--------------+-------------+--------------------+--------------------+-------------------+------+-------------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+-----------------------------+----------------------+--------------------+------------+------------------+------------+--------------------+-------------+----------------+------------------+--------------------+-------------------+-------------------+--------------------+------------------+-------------------+-------------------+--------------------+----------------+-------------------+------------------------+----------+-----------+-----------------

## 2. Estadísticas Básicas del Negocio

In [5]:
print("=" * 70)
print("MÉTRICAS PRINCIPALES DEL NEGOCIO")
print("=" * 70)

# Contar entidades únicas
metrics = df.agg(
    F.countDistinct('order_id').alias('ordenes_unicas'),
    F.countDistinct('customer_id').alias('clientes_unicos'),
    F.countDistinct('seller_id').alias('vendedores_unicos'),
    F.countDistinct('product_id').alias('productos_unicos'),
    F.sum('price').alias('revenue_total'),
    F.avg('price').alias('ticket_promedio'),
    F.avg('freight_value').alias('costo_envio_promedio')
).collect()[0]

print(f"\nÓrdenes únicas: {metrics['ordenes_unicas']:,}")
print(f"Clientes únicos: {metrics['clientes_unicos']:,}")
print(f"Vendedores únicos: {metrics['vendedores_unicos']:,}")
print(f"Productos únicos: {metrics['productos_unicos']:,}")
print(f"\nRevenue total: R$ {metrics['revenue_total']:,.2f}")
print(f"Ticket promedio: R$ {metrics['ticket_promedio']:,.2f}")
print(f"Costo envío promedio: R$ {metrics['costo_envio_promedio']:,.2f}")

MÉTRICAS PRINCIPALES DEL NEGOCIO

Órdenes únicas: 98,666
Clientes únicos: 98,666
Vendedores únicos: 3,095
Productos únicos: 32,951

Revenue total: R$ 13,591,643.70
Ticket promedio: R$ 120.65
Costo envío promedio: R$ 19.99


## 3. Análisis Temporal

In [6]:
print("=" * 70)
print("ANÁLISIS TEMPORAL")
print("=" * 70)

# Rango de fechas
date_range = df.agg(
    F.min('order_purchase_timestamp').alias('fecha_min'),
    F.max('order_purchase_timestamp').alias('fecha_max')
).collect()[0]

print(f"\nRango de fechas: {date_range['fecha_min']} a {date_range['fecha_max']}")

# Ventas por año
print("\nVentas por año:")
ventas_year = df.groupBy('order_year') \
    .agg(
        F.countDistinct('order_id').alias('ordenes'),
        F.sum('price').alias('revenue'),
        F.avg('price').alias('ticket_promedio')
    ) \
    .orderBy('order_year')

ventas_year.show()

ANÁLISIS TEMPORAL

Rango de fechas: 2016-09-04 21:15:19 a 2018-09-03 09:06:57

Ventas por año:
+----------+-------+------------------+------------------+
|order_year|ordenes|           revenue|   ticket_promedio|
+----------+-------+------------------+------------------+
|      2016|    312|49785.919999999955|134.55654054054042|
|      2017|  44579| 6155806.979999649|121.02483052846118|
|      2018|  53775| 7386050.799999507|   120.26264816985|
+----------+-------+------------------+------------------+



In [7]:
# Ventas por mes (últimos 12 meses con datos)
print("Ventas mensuales:")
ventas_mensual = df.groupBy('order_year', 'order_month') \
    .agg(
        F.countDistinct('order_id').alias('ordenes'),
        F.sum('price').alias('revenue')
    ) \
    .orderBy('order_year', 'order_month', ascending=False)

ventas_mensual.show(12)

Ventas mensuales:
+----------+-----------+-------+------------------+
|order_year|order_month|ordenes|           revenue|
+----------+-----------+-------+------------------+
|      2018|          9|      1|             145.0|
|      2018|          8|   6452| 854686.3299999912|
|      2018|          7|   6273|  895507.219999992|
|      2018|          6|   6160| 865124.3099999918|
|      2018|          5|   6853| 996517.6799999913|
|      2018|          4|   6934| 996647.7499999902|
|      2018|          3|   7188| 983213.4399999892|
|      2018|          2|   6694| 844178.7099999895|
|      2018|          1|   7220| 950030.3599999885|
|      2017|         12|   5624| 743914.1699999925|
|      2017|         11|   7451|1010271.3699999888|
|      2017|         10|   4568| 664219.4299999953|
+----------+-----------+-------+------------------+
only showing top 12 rows



In [8]:
# Ventas por día de la semana
print("Ventas por día de la semana (0=Lunes, 6=Domingo):")
ventas_dow = df.groupBy('order_day_of_week') \
    .agg(
        F.countDistinct('order_id').alias('ordenes'),
        F.sum('price').alias('revenue')
    ) \
    .orderBy('order_day_of_week')

ventas_dow.show()

Ventas por día de la semana (0=Lunes, 6=Domingo):
+-----------------+-------+------------------+
|order_day_of_week|ordenes|           revenue|
+-----------------+-------+------------------+
|                0|  16068| 2230812.510000029|
|                1|  15831|2172647.8200000282|
|                2|  15425|2113843.5900000297|
|                3|  14639|2018615.7800000238|
|                4|  14002|1962426.7500000182|
|                5|  10813|1504018.3600000008|
|                6|  11888| 1589278.890000009|
+-----------------+-------+------------------+



## 4. Análisis de Productos

In [9]:
print("=" * 70)
print("ANÁLISIS DE PRODUCTOS")
print("=" * 70)

# Top 20 categorías por ventas
print("\nTop 20 categorías por revenue:")
top_categorias = df.groupBy('product_category_name_english') \
    .agg(
        F.countDistinct('order_id').alias('ordenes'),
        F.sum('price').alias('revenue'),
        F.avg('price').alias('precio_promedio'),
        F.avg('avg_review_score').alias('review_promedio')
    ) \
    .orderBy(F.col('revenue').desc()) \
    .limit(20)

top_categorias.show(truncate=False)

ANÁLISIS DE PRODUCTOS

Top 20 categorías por revenue:
+-----------------------------+-------+------------------+------------------+------------------+
|product_category_name_english|ordenes|revenue           |precio_promedio   |review_promedio   |
+-----------------------------+-------+------------------+------------------+------------------+
|health_beauty                |8836   |1258681.3399999873|130.1635305067205 |4.141826588791545 |
|watches_gifts                |5624   |1205005.6799999953|201.13598397596317|4.018855218855219 |
|bed_bath_table               |9417   |1036988.6799999864|93.2963274853789  |3.898333636860317 |
|sports_leisure               |7720   |988048.9699999891 |114.3442853836349 |4.10703880666589  |
|computers_accessories        |6689   |911954.3199999917 |116.51390315574187|3.9326008738113596|
|furniture_decor              |6449   |729762.4899999945 |87.5644936405081  |3.9073333333333333|
|cool_stuff                   |3632   |635290.8500000003 |167.35796891464

In [10]:
# Estadísticas de dimensiones de productos
print("\nEstadísticas de dimensiones de productos:")
product_stats = df.select(
    'product_weight_g',
    'product_length_cm',
    'product_height_cm',
    'product_width_cm'
).describe()

product_stats.show()


Estadísticas de dimensiones de productos:
+-------+------------------+------------------+------------------+------------------+
|summary|  product_weight_g| product_length_cm| product_height_cm|  product_width_cm|
+-------+------------------+------------------+------------------+------------------+
|  count|            112632|            112632|            112632|            112632|
|   mean| 2093.672047020385|30.153668584416508|16.593765537325094|22.996546274593367|
| stddev|3751.5968835138533|16.153449156461008| 13.44348317091501| 11.70726831380731|
|    min|               0.0|               7.0|               2.0|               6.0|
|    max|           40425.0|             105.0|             105.0|             118.0|
+-------+------------------+------------------+------------------+------------------+



## 5. Análisis Geográfico

In [11]:
print("=" * 70)
print("ANÁLISIS GEOGRÁFICO")
print("=" * 70)

# Top 15 estados por ventas (clientes)
print("\nTop 15 estados por ventas (clientes):")
top_estados = df.groupBy('customer_state') \
    .agg(
        F.countDistinct('order_id').alias('ordenes'),
        F.countDistinct('customer_id').alias('clientes'),
        F.sum('price').alias('revenue'),
        F.avg('distance_km').alias('distancia_promedio_km')
    ) \
    .orderBy(F.col('revenue').desc()) \
    .limit(15)

top_estados.show()

ANÁLISIS GEOGRÁFICO

Top 15 estados por ventas (clientes):
+--------------+-------+--------+------------------+---------------------+
|customer_state|ordenes|clientes|           revenue|distancia_promedio_km|
+--------------+-------+--------+------------------+---------------------+
|            SP|  41375|   41375| 5202955.050001498|    247.2153393563246|
|            RJ|  12762|   12762| 1824092.669999808|   487.30909598640284|
|            MG|  11544|   11544| 1585308.029999873|    533.4248103704546|
|            RS|   5432|    5432| 750304.0200000231|    868.0454619301702|
|            PR|   4998|    4998| 683083.7600000206|    487.3515751197135|
|            SC|   3612|    3612| 520553.3400000074|    572.9596323779759|
|            BA|   3358|    3358| 511349.9900000073|   1345.0605015485894|
|            DF|   2125|    2125| 302603.9399999978|    829.8576880979157|
|            GO|   2007|    2007| 294591.9499999972|    775.1183799385743|
|            ES|   2025|    2025|275037.3

In [12]:
# Top 15 ciudades por ventas
print("\nTop 15 ciudades por ventas:")
top_ciudades = df.groupBy('customer_city', 'customer_state') \
    .agg(
        F.countDistinct('order_id').alias('ordenes'),
        F.sum('price').alias('revenue')
    ) \
    .orderBy(F.col('revenue').desc()) \
    .limit(15)

top_ciudades.show(truncate=False)


Top 15 ciudades por ventas:
+---------------------+--------------+-------+------------------+
|customer_city        |customer_state|ordenes|revenue           |
+---------------------+--------------+-------+------------------+
|sao paulo            |SP            |15402  |1914924.540000007 |
|rio de janeiro       |RJ            |6834   |992538.8599999903 |
|belo horizonte       |MG            |2750   |355611.1300000004 |
|brasilia             |DF            |2116   |301920.25000000047|
|curitiba             |PR            |1510   |211738.0600000005 |
|porto alegre         |RS            |1372   |190562.08000000037|
|campinas             |SP            |1429   |187844.53000000038|
|salvador             |BA            |1238   |181104.4200000004 |
|guarulhos            |SP            |1178   |144268.3900000003 |
|niteroi              |RJ            |845    |117907.12000000008|
|goiania              |GO            |687    |106111.17000000004|
|sao bernardo do campo|SP            |928    |1

In [13]:
# Análisis de distancia entre cliente y vendedor
print("\nEstadísticas de distancia cliente-vendedor:")
distance_stats = df.select('distance_km').describe()
distance_stats.show()

# Distribución por rangos de distancia
print("\nDistribución por rangos de distancia:")
distance_ranges = df.withColumn(
    'rango_distancia',
    F.when(F.col('distance_km') < 100, '0-100 km')
     .when(F.col('distance_km') < 500, '100-500 km')
     .when(F.col('distance_km') < 1000, '500-1000 km')
     .when(F.col('distance_km') < 2000, '1000-2000 km')
     .otherwise('2000+ km')
).groupBy('rango_distancia') \
 .agg(
     F.count('*').alias('ordenes'),
     F.avg('freight_value').alias('costo_envio_promedio')
 ) \
 .orderBy('rango_distancia')

distance_ranges.show()


Estadísticas de distancia cliente-vendedor:
+-------+-----------------+
|summary|      distance_km|
+-------+-----------------+
|  count|           112096|
|   mean|596.9591827347342|
| stddev|589.9788204712339|
|    min|              0.0|
|    max|8677.911621858064|
+-------+-----------------+


Distribución por rangos de distancia:
+---------------+-------+--------------------+
|rango_distancia|ordenes|costo_envio_promedio|
+---------------+-------+--------------------+
|       0-100 km|  20998|  11.757495951995702|
|     100-500 km|  43324|  18.400342535313087|
|   1000-2000 km|  11269|   29.20318040642463|
|       2000+ km|   6980|    34.7602206303726|
|    500-1000 km|  30079|  21.148715715281412|
+---------------+-------+--------------------+



## 6. Análisis de Pagos

In [14]:
print("=" * 70)
print("ANÁLISIS DE PAGOS")
print("=" * 70)

# Distribución de tipos de pago
print("\nDistribución de tipos de pago:")
payment_types = df.groupBy('payment_type') \
    .agg(
        F.countDistinct('order_id').alias('ordenes'),
        F.sum('payment_value').alias('valor_total'),
        F.avg('payment_value').alias('valor_promedio')
    ) \
    .orderBy(F.col('ordenes').desc())

payment_types.show()

ANÁLISIS DE PAGOS

Distribución de tipos de pago:
+------------+-------+--------------------+------------------+
|payment_type|ordenes|         valor_total|    valor_promedio|
+------------+-------+--------------------+------------------+
| credit_card|  75623|1.5744515519999808E7|183.03957961798025|
|      boleto|  19614|   4059699.599999999|177.53529540385705|
|     voucher|   1908|   250435.7299999998|120.75011089681765|
|  debit_card|   1520|           253483.86| 150.0792539964476|
|        NULL|      1|                NULL|              NULL|
+------------+-------+--------------------+------------------+



In [15]:
# Análisis de cuotas (installments)
print("\nAnálisis de cuotas de pago:")
installments = df.groupBy('payment_installments') \
    .agg(
        F.countDistinct('order_id').alias('ordenes'),
        F.avg('payment_value').alias('valor_promedio')
    ) \
    .orderBy('payment_installments') \
    .limit(15)

installments.show()


Análisis de cuotas de pago:
+--------------------+-------+------------------+
|payment_installments|ordenes|    valor_promedio|
+--------------------+-------+------------------+
|                NULL|      1|              NULL|
|                 0.0|      2|            106.19|
|                 1.0|  47823| 142.6267617138192|
|                 2.0|  12291|136.53734650567168|
|                 3.0|  10371| 155.6109657055113|
|                 4.0|   7022| 180.4092258308157|
|                 5.0|   5188|199.06735911970645|
|                 6.0|   3887|236.91939808778793|
|                 7.0|   1611| 195.3237006578948|
|                 8.0|   4237|312.66205718824455|
|                 9.0|    635| 205.1770661157025|
|                10.0|   5261|435.17455570194727|
|                11.0|     22|124.53079999999999|
|                12.0|    131|325.61239263803674|
|                13.0|     16| 148.6772222222222|
+--------------------+-------+------------------+



## 7. Análisis de Reviews y Satisfacción

In [16]:
print("=" * 70)
print("ANÁLISIS DE REVIEWS Y SATISFACCIÓN")
print("=" * 70)

# Distribución de scores de review
print("\nDistribución de review scores:")
review_distribution = df.filter(F.col('avg_review_score').isNotNull()) \
    .withColumn('review_score_rounded', F.round('avg_review_score')) \
    .groupBy('review_score_rounded') \
    .agg(
        F.count('*').alias('ordenes'),
        F.avg('payment_value').alias('valor_promedio')
    ) \
    .orderBy('review_score_rounded')

review_distribution.show()

# Estadísticas generales de reviews
review_stats = df.filter(F.col('avg_review_score').isNotNull()) \
    .agg(
        F.avg('avg_review_score').alias('review_promedio_general'),
        F.count('*').alias('total_ordenes_con_review')
    ).collect()[0]

print(f"\nReview promedio general: {review_stats['review_promedio_general']:.2f}")
print(f"Órdenes con review: {review_stats['total_ordenes_con_review']:,}")

ANÁLISIS DE REVIEWS Y SATISFACCIÓN

Distribución de review scores:
+--------------------+-------+------------------+
|review_score_rounded|ordenes|    valor_promedio|
+--------------------+-------+------------------+
|                 1.0|  14079|242.99848323387002|
|                 2.0|   3851|202.59349259932466|
|                 3.0|   9430|175.15183987274685|
|                 4.0|  21198| 170.9710637796009|
|                 5.0|  63150| 167.8964405383943|
+--------------------+-------+------------------+


Review promedio general: 4.03
Órdenes con review: 111,708


In [17]:
# Categorías con mejores y peores reviews
print("\nTop 10 categorías con MEJORES reviews:")
best_reviews = df.filter(F.col('avg_review_score').isNotNull()) \
    .groupBy('product_category_name_english') \
    .agg(
        F.avg('avg_review_score').alias('review_promedio'),
        F.count('*').alias('cantidad_ordenes')
    ) \
    .filter(F.col('cantidad_ordenes') >= 100) \
    .orderBy(F.col('review_promedio').desc()) \
    .limit(10)

best_reviews.show(truncate=False)

print("\nTop 10 categorías con PEORES reviews:")
worst_reviews = df.filter(F.col('avg_review_score').isNotNull()) \
    .groupBy('product_category_name_english') \
    .agg(
        F.avg('avg_review_score').alias('review_promedio'),
        F.count('*').alias('cantidad_ordenes')
    ) \
    .filter(F.col('cantidad_ordenes') >= 100) \
    .orderBy(F.col('review_promedio').asc()) \
    .limit(10)

worst_reviews.show(truncate=False)


Top 10 categorías con MEJORES reviews:
+-----------------------------+-----------------+----------------+
|product_category_name_english|review_promedio  |cantidad_ordenes|
+-----------------------------+-----------------+----------------+
|books_general_interest       |4.446265938069216|549             |
|books_technical              |4.363636363636363|264             |
|luggage_accessories          |4.315257352941177|1088            |
|food_drink                   |4.312274368231047|277             |
|fashion_shoes                |4.238372093023256|258             |
|food                         |4.218181818181818|495             |
|stationery                   |4.195478191276511|2499            |
|pet_shop                     |4.184305627258647|1937            |
|computers                    |4.175            |200             |
|home_appliances              |4.166449652777778|768             |
+-----------------------------+-----------------+----------------+


Top 10 categorías co

## 8. Análisis de Entregas

In [18]:
print("=" * 70)
print("ANÁLISIS DE ENTREGAS")
print("=" * 70)

# Estadísticas de tiempos de entrega
print("\nEstadísticas de tiempo de entrega:")
delivery_stats = df.filter(F.col('total_delivery_time_days').isNotNull()) \
    .select('total_delivery_time_days', 'delivery_delay_days') \
    .describe()

delivery_stats.show()

# Porcentaje de entregas a tiempo vs retrasadas
delivery_performance = df.filter(F.col('delivery_delay_days').isNotNull()) \
    .withColumn(
        'status_entrega',
        F.when(F.col('delivery_delay_days') <= 0, 'A tiempo o anticipada')
         .otherwise('Retrasada')
    ) \
    .groupBy('status_entrega') \
    .agg(
        F.count('*').alias('ordenes'),
        F.avg('avg_review_score').alias('review_promedio')
    )

print("\nPerformance de entregas:")
delivery_performance.show()

ANÁLISIS DE ENTREGAS

Estadísticas de tiempo de entrega:
+-------+------------------------+-------------------+
|summary|total_delivery_time_days|delivery_delay_days|
+-------+------------------------+-------------------+
|  count|                  110196|             110196|
|   mean|      12.007722603361284|-12.030200733238956|
| stddev|       9.451455432008565|  10.16015653052225|
|    min|                     0.0|             -147.0|
|    max|                   209.0|              188.0|
+-------+------------------------+-------------------+


Performance de entregas:
+--------------------+-------+-----------------+
|      status_entrega|ordenes|  review_promedio|
+--------------------+-------+-----------------+
|A tiempo o antici...| 102931|4.208267748610908|
|           Retrasada|   7265|2.257411067193676|
+--------------------+-------+-----------------+



In [19]:
# Relación entre distancia y tiempo de entrega
print("\nRelación entre distancia y tiempo de entrega:")
distance_delivery = df.filter(
    (F.col('distance_km').isNotNull()) & 
    (F.col('total_delivery_time_days').isNotNull())
).select(
    F.corr('distance_km', 'total_delivery_time_days').alias('correlacion')
)

distance_delivery.show()

# Tiempo de entrega por rango de distancia
print("\nTiempo promedio de entrega por rango de distancia:")
delivery_by_distance = df.filter(
    (F.col('distance_km').isNotNull()) & 
    (F.col('total_delivery_time_days').isNotNull())
).withColumn(
    'rango_distancia',
    F.when(F.col('distance_km') < 100, '0-100 km')
     .when(F.col('distance_km') < 500, '100-500 km')
     .when(F.col('distance_km') < 1000, '500-1000 km')
     .when(F.col('distance_km') < 2000, '1000-2000 km')
     .otherwise('2000+ km')
).groupBy('rango_distancia') \
 .agg(
     F.avg('total_delivery_time_days').alias('dias_entrega_promedio'),
     F.avg('delivery_delay_days').alias('retraso_promedio_dias'),
     F.count('*').alias('ordenes')
 ) \
 .orderBy('rango_distancia')

delivery_by_distance.show()


Relación entre distancia y tiempo de entrega:
+------------------+
|       correlacion|
+------------------+
|0.3938105986746739|
+------------------+


Tiempo promedio de entrega por rango de distancia:
+---------------+---------------------+---------------------+-------+
|rango_distancia|dias_entrega_promedio|retraso_promedio_dias|ordenes|
+---------------+---------------------+---------------------+-------+
|       0-100 km|    6.068453688784727|   -9.581856354670181|  20481|
|     100-500 km|   10.959983031674208|  -12.266167043740573|  42432|
|   1000-2000 km|    17.43559952800218|  -13.064082781156396|  11017|
|       2000+ km|   20.634103304459416|  -12.834616618543471|   6234|
|    500-1000 km|   13.756983997830215|  -12.855675345809601|  29496|
+---------------+---------------------+---------------------+-------+



## 9. Análisis de Vendedores

In [20]:
print("=" * 70)
print("ANÁLISIS DE VENDEDORES")
print("=" * 70)

# Top 20 vendedores por ventas
print("\nTop 20 vendedores por revenue:")
top_sellers = df.groupBy('seller_id') \
    .agg(
        F.countDistinct('order_id').alias('ordenes'),
        F.sum('price').alias('revenue'),
        F.avg('price').alias('ticket_promedio'),
        F.avg('avg_review_score').alias('review_promedio')
    ) \
    .orderBy(F.col('revenue').desc()) \
    .limit(20)

top_sellers.show(truncate=False)

ANÁLISIS DE VENDEDORES

Top 20 vendedores por revenue:
+--------------------------------+-------+------------------+------------------+------------------+
|seller_id                       |ordenes|revenue           |ticket_promedio   |review_promedio   |
+--------------------------------+-------+------------------+------------------+------------------+
|4869f7a5dfa277a7dca6462dcf3b52b2|1132   |229472.6300000003 |198.5057352941179 |4.122822299651568 |
|53243585a1d6dc2643021fd1853d8905|358    |222776.05000000005|543.3562195121953 |4.075980392156863 |
|4a3ca9315b744ce9f8e9374361493884|1806   |200472.92000000048|100.89225968797206|3.803261977573904 |
|fa1c13f2614d7b5c4749cbc52fecda94|585    |194042.0300000002 |331.1297440273041 |4.34020618556701  |
|7c67e1448b00f6e969d365cea6b010ab|982    |187923.89000000013|137.7741129032259 |3.3414454277286136|
|7e93a43ef30c4f03f38b393420bc753a|336    |176431.8700000001 |518.9172647058826 |4.206489675516224 |
|da8622b14eb17ae2831f4ac5b9dab84a|1314   |160

In [21]:
# Distribución de vendedores por estado
print("\nVendedores por estado:")
sellers_by_state = df.groupBy('seller_state') \
    .agg(
        F.countDistinct('seller_id').alias('cantidad_vendedores'),
        F.sum('price').alias('revenue_total')
    ) \
    .orderBy(F.col('revenue_total').desc()) \
    .limit(15)

sellers_by_state.show()


Vendedores por estado:
+------------+-------------------+------------------+
|seller_state|cantidad_vendedores|     revenue_total|
+------------+-------------------+------------------+
|          SP|               1849|  8753396.21000002|
|          PR|                349|1261887.2100000007|
|          MG|                244|1011564.7400000015|
|          RJ|                171| 843984.2199999994|
|          SC|                190| 632426.0700000002|
|          RS|                129|378559.54000000004|
|          BA|                 19|285561.55999999965|
|          DF|                 30| 97749.48000000001|
|          PE|                  9| 91493.85000000034|
|          GO|                 40| 66399.20999999999|
|          ES|                 23| 47689.61000000002|
|          MA|                  1| 36408.95000000018|
|          CE|                 13|20240.640000000007|
|          PB|                  6|           17095.0|
|          MT|                  4|          17070.72|
+---

## 10. Correlaciones entre Variables Numéricas

In [22]:
print("=" * 70)
print("CORRELACIONES ENTRE VARIABLES")
print("=" * 70)

# Variables numéricas clave
numeric_cols = [
    'price',
    'freight_value',
    'payment_value',
    'avg_review_score',
    'distance_km',
    'total_delivery_time_days',
    'delivery_delay_days'
]

# Calcular correlaciones importantes
print("\nCorrelaciones importantes:")

correlations = [
    ('price', 'avg_review_score', 'Precio vs Review'),
    ('distance_km', 'freight_value', 'Distancia vs Costo Envío'),
    ('distance_km', 'total_delivery_time_days', 'Distancia vs Tiempo Entrega'),
    ('delivery_delay_days', 'avg_review_score', 'Retraso vs Review'),
    ('payment_value', 'avg_review_score', 'Valor Pago vs Review'),
    ('freight_value', 'avg_review_score', 'Costo Envío vs Review')
]

for col1, col2, label in correlations:
    corr = df.filter(
        (F.col(col1).isNotNull()) & (F.col(col2).isNotNull())
    ).select(
        F.corr(col1, col2).alias('correlacion')
    ).collect()[0]['correlacion']
    
    print(f"{label}: {corr:.4f}")

CORRELACIONES ENTRE VARIABLES

Correlaciones importantes:
Precio vs Review: -0.0041
Distancia vs Costo Envío: 0.3894
Distancia vs Tiempo Entrega: 0.3938
Retraso vs Review: -0.2293
Valor Pago vs Review: -0.0846
Costo Envío vs Review: -0.0364


## 11. Resumen Ejecutivo

In [23]:
print("=" * 70)
print("RESUMEN EJECUTIVO - INSIGHTS CLAVE")
print("=" * 70)

# Recopilar insights clave
total_orders = df.select(F.countDistinct('order_id')).collect()[0][0]
total_revenue = df.select(F.sum('price')).collect()[0][0]
avg_review = df.filter(F.col('avg_review_score').isNotNull()).select(F.avg('avg_review_score')).collect()[0][0]
on_time_pct = df.filter(F.col('delivery_delay_days').isNotNull()).filter(F.col('delivery_delay_days') <= 0).count() / df.filter(F.col('delivery_delay_days').isNotNull()).count() * 100

print(f"\n📊 MÉTRICAS GENERALES")
print(f"  • Total de órdenes: {total_orders:,}")
print(f"  • Revenue total: R$ {total_revenue:,.2f}")
print(f"  • Review promedio: {avg_review:.2f}/5.0")
print(f"  • % Entregas a tiempo: {on_time_pct:.1f}%")

print(f"\n🎯 INSIGHTS CLAVE PARA MODELADO PREDICTIVO:")
print(f"  1. La satisfacción del cliente (reviews) está influenciada por:")
print(f"     - Tiempo de entrega y retrasos")
print(f"     - Categoría de producto")
print(f"     - Distancia entre cliente y vendedor")
print(f"\n  2. Factores que impactan el tiempo de entrega:")
print(f"     - Distancia geográfica (correlación significativa)")
print(f"     - Estado de origen y destino")
print(f"\n  3. Patrones de compra:")
print(f"     - Variación por mes y día de la semana")
print(f"     - Concentración en estados específicos (SP, RJ, MG)")
print(f"     - Preferencia por tipos de pago específicos")
print(f"\n  4. Oportunidades para modelos predictivos:")
print(f"     - Predicción de review score (clasificación/regresión)")
print(f"     - Predicción de tiempo de entrega (regresión)")
print(f"     - Segmentación de clientes (clustering)")
print(f"     - Detección de riesgo de retraso (clasificación)")
print(f"     - Recomendación de productos (collaborative filtering)")

print("\n" + "=" * 70)
print("✓ Análisis exploratorio completado")
print("=" * 70)

RESUMEN EJECUTIVO - INSIGHTS CLAVE

📊 MÉTRICAS GENERALES
  • Total de órdenes: 98,666
  • Revenue total: R$ 13,591,643.70
  • Review promedio: 4.03/5.0
  • % Entregas a tiempo: 93.4%

🎯 INSIGHTS CLAVE PARA MODELADO PREDICTIVO:
  1. La satisfacción del cliente (reviews) está influenciada por:
     - Tiempo de entrega y retrasos
     - Categoría de producto
     - Distancia entre cliente y vendedor

  2. Factores que impactan el tiempo de entrega:
     - Distancia geográfica (correlación significativa)
     - Estado de origen y destino

  3. Patrones de compra:
     - Variación por mes y día de la semana
     - Concentración en estados específicos (SP, RJ, MG)
     - Preferencia por tipos de pago específicos

  4. Oportunidades para modelos predictivos:
     - Predicción de review score (clasificación/regresión)
     - Predicción de tiempo de entrega (regresión)
     - Segmentación de clientes (clustering)
     - Detección de riesgo de retraso (clasificación)
     - Recomendación de prod

---

## Próximos Pasos

Con este análisis exploratorio completado, ahora podemos proceder a:

1. **Feature Engineering avanzado**: Crear features adicionales basadas en los insights descubiertos
2. **Modelado Predictivo**: Implementar modelos de ML usando MLlib de Spark
3. **Evaluación y Optimización**: Ajustar hiperparámetros y evaluar performance
4. **Deployment**: Preparar el modelo para producción